In [23]:
%%capture
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [24]:
%%capture
import sys
pip install autogluon
import autogluon as ag
from autogluon.text import TextPredictor

SyntaxError: invalid syntax (4054218019.py, line 2)

In [3]:
%%capture
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv',index_col ='id')
print(train.shape)
test = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv',index_col = 'id')
print(test.shape)

(36473, 4)
(36, 3)


In [5]:
print(train.head())

                     anchor                  target context  score
id                                                                
37d61fd2272659b1  abatement  abatement of pollution     A47   0.50
7b9652b17b68b7a4  abatement          act of abating     A47   0.75
36d72442aefd8232  abatement         active catalyst     A47   0.25
5296b0c19e1ce60e  abatement     eliminating process     A47   0.50
54c1e3b9184cb5b6  abatement           forest region     A47   0.00


In [6]:
print(test.head())

                              anchor                         target context
id                                                                         
4112d61851461f60            opc drum  inorganic photoconductor drum     G02
09e418c93a776564     adjust gas flow              altering gas flow     F23
36baf228038e314b      lower trunnion                 lower locating     B60
1f37ead645e7f0c8       cap component                  upper portion     D06
71a5b6ad068d531f  neural stimulation      artificial neural network     H04


In [7]:
%%capture
codes = pd.read_csv('../input/cpc-titles/title_info.csv')
print(codes.head())

In [8]:
codes_copy = codes.copy()
codes = codes.rename(columns = {"code" : "context"})
display(codes)


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0
...,...,...,...,...,...,...,...
260471,Y10T483/1864,including tool pot or adapter,Y,10.0,T,483.0,1864.0
260472,Y10T483/1873,Indexing matrix,Y,10.0,T,483.0,1873.0
260473,Y10T483/1882,Rotary disc,Y,10.0,T,483.0,1882.0
260474,Y10T483/1891,Chain or belt,Y,10.0,T,483.0,1891.0


In [9]:
context = train['context'].unique()
print(context.shape)


(106,)


In [10]:
target = train['target'].unique()
print(target.shape)

(29340,)


In [11]:
train2 = pd.merge(train, codes[["context","title"]], on= "context", how = "left")
print(train2.head())

      anchor                  target context  score  \
0  abatement  abatement of pollution     A47   0.50   
1  abatement          act of abating     A47   0.75   
2  abatement         active catalyst     A47   0.25   
3  abatement     eliminating process     A47   0.50   
4  abatement           forest region     A47   0.00   

                                               title  
0  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
1  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
2  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
3  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
4  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  


In [12]:
np.random.seed(123)

#display(test)

train.drop(columns = ['context'],inplace = True)

print(train.head(3))



                     anchor                  target  score
id                                                        
37d61fd2272659b1  abatement  abatement of pollution   0.50
7b9652b17b68b7a4  abatement          act of abating   0.75
36d72442aefd8232  abatement         active catalyst   0.25


In [13]:
print("Min Score = ", min(train2['score']), "Max Score = ", max(train2['score']))

Min Score =  0.0 Max Score =  1.0


In [14]:
predictor = TextPredictor(label = 'score', path='./ag_sst')
predictor.fit(train, time_limit=60)

NameError: name 'TextPredictor' is not defined

In [ ]:
predictor = TextPredictor.load(path = "./ag_sst/")



In [ ]:
train_score = predictor.evaluate(train.sample(2000), metrics=['rmse', 'pearsonr', 'spearmanr'])

print('RMSE = {:.2f}'.format(train_score['rmse']))
print('PEARSONR = {:.4f}'.format(train_score['pearsonr']))
print('SPEARMANR = {:.4f}'.format(train_score['spearmanr']))



In [ ]:
results = predictor.predict(test)
results

In [ ]:
display(test)


In [ ]:
submission = pd.DataFrame({ 'score': results})
submission.to_csv('submission.csv', index=True)

In [ ]:
submission

In [ ]:
# os.chdir("./")
# !touch config.yaml
# !ls